In [1]:
# importing the required libraries
import numpy as np 
import pandas as pd
import seaborn as sns  
import numpy as np 
from matplotlib import pyplot as plt 

# DATA PREPROCESSING

In [2]:
# reading the input file
df = pd.read_csv("C:\\Users\\vvsat\\Documents\\machine learning\\blood glucose challenge\\dataset_0.csv")
print(df.head()) 

    TIME  FOOD  RAPI  LAI  BG
0  00:00   NaN   NaN  NaN NaN
1  00:10   NaN   NaN  NaN NaN
2  00:20   NaN   NaN  NaN NaN
3  00:30   NaN   NaN  NaN NaN
4  00:40   NaN   NaN  NaN NaN


In [3]:
# dropping the 'TIME' column
df = df.drop("TIME", axis=1)
#df = df.dropna()

In [ ]:
# printing the head of the final dataframe
print(df.head())

In [ ]:
# plotting the data
print("VISUALISING THE INDEPENDENT RELATIONSHIPS")
 
sns.lmplot(x='FOOD', y='BG', data=df)

sns.lmplot(x='RAPI', y='BG', data=df)  

sns.lmplot(x='LAI', y='BG', data=df)  

In [ ]:
# displaying the correlation between the given features
corr = df.corr()
print(corr)
sns.heatmap(corr, 
         xticklabels=corr.columns, 
         yticklabels=corr.columns)

In [ ]:
# understanding the skewness and kurtosis of the data

print("AFTER LOG TRANSFORM) \n ")
print("SKEWNESS : \n",df.skew(axis = 0, skipna = True))
print("\n \n KURTOSIS : \n",df.kurtosis(axis = 0, skipna = True))

In [ ]:
# applying logrithmic transform for BG
# applied only to the targets

df['BG'] = np.log(df['BG']) 

In [ ]:
# food, rapi and lai - independent variables
x_df = df[['FOOD', 'RAPI', 'LAI']]

# BG - dependent/predictor variable
y_df = df['BG'] 
  
x_df = x_df.to_numpy()
y_df = y_df.to_numpy()

In [ ]:
from sklearn.impute import SimpleImputer
# imputer
imputer = SimpleImputer(missing_values=np.nan, strategy="mean", add_indicator=True)
imputer.fit(x_df)

x = imputer.transform(x_df)
y = y_df.reshape(-1, 1)
imputer.fit(y)
y = imputer.transform(y)

In [ ]:
# scaling the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
scaler = StandardScaler()
scaler.fit(x)

# scaling the data with a mean value 0 and standard deviation of 1
x = scaler.transform(x) 

In [ ]:
# splitting the data into train and test(20%) sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# LINEAR REGRESSION

In [ ]:
from sklearn import linear_model

# Create Linear Regression object
model_lrg = linear_model.LinearRegression(fit_intercept=True, normalize='deprecated')

model_lrg.fit(X_train, y_train) #fitting the model 

print('R^2 value =', model_lrg.score(X_train, y_train))  #Printing the R^2 value     

In [ ]:
# testing the model 
prediction_test = model_lrg.predict(X_test)    
print(y_test)
print(prediction_test)
print("Mean square error =", np.mean(prediction_test-y_test)**2)

In [ ]:
print('regression coefficient =', model_lrg.coef_, '\n','regression intercept =', model_lrg.intercept_)

# LASSO

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [ ]:
# defining the lasso model
lasso = Lasso(random_state=None,  fit_intercept=True, normalize='deprecated', max_iter=100000)

In [ ]:
# grid search
alphas = np.logspace(-4, -0.5, 30) # intercept

# tuning alpha
tuned_parameters = [{"alpha": alphas}]
n_folds = 3

model_lasso = GridSearchCV(lasso, tuned_parameters, cv=n_folds, refit=True)

# refitting the lasso model with the tuned parameters
model_lasso  = model_lasso.fit(X_train, y_train)

print('R^2 value =', model_lasso.score(X_train, y_train))  #Printing the R^2 value

In [ ]:
# displaying the mean and standard deviation of the samples
scores = model_lasso.cv_results_["mean_test_score"]
scores_std = model_lasso.cv_results_["std_test_score"]
plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores) 

In [ ]:
# plot error lines showing +/- std. errors of the scores
std_error = scores_std / np.sqrt(n_folds)

plt.semilogx(alphas, scores + std_error, "b--")
plt.semilogx(alphas, scores - std_error, "b--")

# alpha=0.2 controls the translucency of the fill color
plt.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)

plt.ylabel("CV score +/- std error")
plt.xlabel("alpha")

# printing the maximum of the scores calculated
plt.axhline(np.max(scores), linestyle="--", color=".3")
plt.xlim([alphas[0], alphas[-1]])

In [ ]:
# testing the model 
prediction_test = model_lasso.predict(X_test)    

print(y_test, prediction_test)
print("Mean square error =", np.mean(prediction_test-y_test)**2)

# RIDGE LINEAR MODEL

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV

In [ ]:
# defining the lasso model
ridge = Ridge(random_state=None,  fit_intercept=True, normalize='deprecated')

In [ ]:
# grid search
alphas = np.logspace(-4, -0.5, 30) # intercept

# tuning alpha
tuned_parameters = [{"alpha": alphas}]
n_folds = 3

model_ridge = GridSearchCV(ridge, tuned_parameters, cv=n_folds, refit=True)

# refitting the lasso model with the tuned parameters
model_ridge  = model_ridge.fit(X_train, y_train)

print('R^2 value =', model_ridge.score(X_train, y_train))  #Printing the R^2 value

In [ ]:
# displaying the mean and standard deviation of the samples
scores = model_ridge.cv_results_["mean_test_score"]
scores_std = model_ridge.cv_results_["std_test_score"]
plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores)

In [ ]:
# plot error lines showing +/- std. errors of the scores
std_error = scores_std / np.sqrt(n_folds)

plt.semilogx(alphas, scores + std_error, "b--")
plt.semilogx(alphas, scores - std_error, "b--")

# alpha=0.2 controls the translucency of the fill color
plt.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)

plt.ylabel("CV score +/- std error")
plt.xlabel("alpha")

# printing the maximum of the scores calculated
plt.axhline(np.max(scores), linestyle="--", color=".3")
plt.xlim([alphas[0], alphas[-1]])

In [ ]:
# testing the model 
prediction_test = model_ridge.predict(X_test)    

print(y_test, prediction_test)
print("Mean square error =", np.mean(prediction_test-y_test)**2)

# POLYNOMIAL REGRESSION

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [ ]:
# polynomial features
for degree in [2, 3, 4]:
    model_pf = make_pipeline(PolynomialFeatures(degree), Ridge(alpha=1e-3))
    model_pf.fit(X_train, y_train)
    
    print('R^2 value for degree', degree, '=', model_pf.score(X_train, y_train))  #Printing the R^2 value

In [ ]:
degree = 4
model_pf = make_pipeline(PolynomialFeatures(degree), Ridge(alpha=1e-3))
model_pf.fit(X_train, y_train)      

print('R^2 value for degree',degree,'=', model_pf.score(X_train, y_train))  #Printing the R^2 value

In [ ]:
# testing the model 
prediction_test = model_pf.predict(X_test)    
print(y_test)
print(prediction_test)
print("Mean square error =", np.mean(prediction_test-y_test)**2)

In [ ]:
from sklearn.ensemble import StackingRegressor

estimator_1 = [
    ("linear regression", model_lrg), 
    ("Polynomial Regression", model_pf)
]

stacking_regressor_1 = StackingRegressor(estimators=estimator_1)
stacking_regressor_1 

# VISUALISING THE RESULTS

In [ ]:
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, cross_val_predict


def plot_regression_results(ax, y_true, y_pred, title, scores, elapsed_time):
    """Scatter plot of the predicted vs true targets."""
    ax.plot(
        [y_true.min(), y_true.max()], [y_true.min(), y_true.max()], "--r", linewidth=2
    )
    ax.scatter(y_true, y_pred, alpha=0.2)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines["left"].set_position(("outward", 10))
    ax.spines["bottom"].set_position(("outward", 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel("Measured")
    ax.set_ylabel("Predicted")
    extra = plt.Rectangle(
        (0, 0), 0, 0, fc="w", fill=False, edgecolor="none", linewidth=0
    )
    ax.legend([extra], [scores], loc="upper left")
    title = title + "\n Evaluation in {:.2f} seconds".format(elapsed_time)
    ax.set_title(title)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 7))
axs = np.ravel(axs)

for ax, (name, est) in zip(
    axs, estimator_1 + [("Stacking Regressor", stacking_regressor_1)]
):
    start_time = time.time()
    score = cross_validate(
        est, X_test, y_test, scoring=["r2", "neg_mean_absolute_error"], n_jobs=2, verbose=0
    )
    elapsed_time = time.time() - start_time

    y_pred = cross_val_predict(est, X_test, y_test, n_jobs=2, verbose=0)

    plot_regression_results(
        ax,
        y_test,
        y_pred,
        name,
        (r"$R^2={:.2f} \pm {:.2f}$" + "\n" + r"$MAE={:.2f} \pm {:.2f}$").format(
            np.mean(score["test_r2"]),
            np.std(score["test_r2"]),
            -np.mean(score["test_neg_mean_absolute_error"]),
            np.std(score["test_neg_mean_absolute_error"]),
        ),
        elapsed_time,
    )
    
plt.suptitle("Single predictors versus stacked predictors \n")
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()